# 📊 Анализ авиаперелетов
## Дашборд в Apache Superset
**Цель:** Создание аналитической витрины и интерактивного дашборда для анализа продаж авиабилетов

In [2]:
import pandas as pd
import psycopg2
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

In [3]:
def run_query(sql, database_name, password, params=None):
    """Выполняет SQL запрос и возвращает DataFrame"""
    try:
        # Создание новое соединения для каждого запроса
        with psycopg2.connect(
            host='localhost',
            port=5432,
            database=database_name, 
            user='postgres',
            password=password
        ) as conn:
            return pd.read_sql_query(sql, conn, params=params)
    except Exception as e:
        print(f'❌ Ошибка выполнения запроса: {e}')
        return None

def execute_sql(sql, database_name, password):
    """Выполняет SQL команду (CREATE, INSERT, etc.)"""
    try:
        with psycopg2.connect(
            host='localhost',
            port=5432,
            database=database_name, 
            user='postgres',
            password=password
        ) as conn:
            with conn.cursor() as cursor:
                cursor.execute(sql)
            conn.commit()
        return True
    except Exception as e:
        print(f"❌ Ошибка выполнения SQL: {e}")
        return False

# Тестирование подключения
database_name = input('Введи имя БД: ')
password = input('Введи пароль: ')
db_info = run_query('SELECT current_database(), current_user, version();', database_name, password)
print('Текущее подключение:')
if db_info is not None:
    display(db_info)

# Все таблицы во всех схемах
all_tables = run_query("""
    SELECT 
        schemaname as schema,
        tablename as table,
        tableowner as owner
    FROM pg_tables 
    WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
    ORDER BY schemaname, tablename;
""", database_name, password)

print('Все таблицы в базе данных:')
if all_tables is None or all_tables.empty:
    print('❌ В базе нет пользовательских таблиц или произошла ошибка')
else:
    display(all_tables)

Введи имя БД:  demo
Введи пароль:  123456


Текущее подключение:


,current_database,current_user,version
0,demo,postgres,"PostgreSQL 18.0 on x86_64-windows, compiled by..."


Все таблицы в базе данных:


,schema,table,owner
0,bookings,aircrafts_data,postgres
1,bookings,airports_data,postgres
2,bookings,boarding_passes,postgres
3,bookings,bookings,postgres
4,bookings,flights,postgres
5,bookings,seats,postgres
6,bookings,ticket_flights,postgres
7,bookings,tickets,postgres
8,dashboards,daily_sales,postgres
9,dashboards,flight_analytics,postgres


In [17]:
# SQL-скрипт для создания витрины
create_sql = """
DROP TABLE IF EXISTS dashboards.flight_analytics;

CREATE TABLE dashboards.flight_analytics AS
SELECT
    -- Даты и время
    DATE(b.book_date) AS sale_date,
    b.book_date AS sale_datetime,
    
    -- Билеты и цены
    tf.fare_conditions AS ticket_class,
    tf.amount AS ticket_amount,
    
    -- Рейсы и направления
    f.flight_id,
    f.departure_airport,
    dep.city->>'en' AS departure_city,
    dep.airport_name->>'en' AS departure_airport_name,
    f.arrival_airport,
    arr.city->>'en' AS arrival_city,
    arr.airport_name->>'en' AS arrival_airport_name,
    f.departure_airport || ' → ' || f.arrival_airport AS direction,
    (dep.city->>'en') || ' → ' || (arr.city->>'en') AS direction_cities,
    f.scheduled_departure,
    f.scheduled_arrival,
    
    -- Самолеты
    f.aircraft_code,
    ad.model->>'en' AS aircraft_model,
    
    -- Статусы
    f.status AS flight_status
    
FROM bookings.bookings b
    JOIN bookings.tickets t ON b.book_ref = t.book_ref
    JOIN bookings.ticket_flights tf ON t.ticket_no = tf.ticket_no 
    JOIN bookings.flights f ON tf.flight_id = f.flight_id 
    JOIN bookings.aircrafts_data ad ON f.aircraft_code = ad.aircraft_code
    JOIN bookings.airports_data dep ON f.departure_airport = dep.airport_code
    JOIN bookings.airports_data arr ON f.arrival_airport = arr.airport_code

ORDER BY sale_date;

-- Проверка данных
SELECT COUNT(*) AS total_rows FROM dashboards.flight_analytics;
SELECT direction, direction_cities, COUNT(*) 
FROM dashboards.flight_analytics 
GROUP BY direction, direction_cities 
LIMIT 10;
"""

# Создание витрины
result = execute_sql(create_sql, database_name, password)
if result:
    print('✅ Витрина данных создана!')
    
    # Кол-во строк
    count_result = run_query('SELECT COUNT(*) AS total_rows FROM dashboards.flight_analytics;', database_name, password)
    if count_result is not None:
        print(f"📊 Всего строк в витрине: {count_result['total_rows'].iloc[0]}")
        
    # Структура витрины
    print('\n📋 Структура витрины flight_analytics:')
    vm_structure = run_query("""
        SELECT 
            column_name,
            data_type,
            is_nullable
        FROM information_schema.columns 
        WHERE table_schema = 'dashboards' 
            AND table_name = 'flight_analytics'
        ORDER BY ordinal_position;
    """, database_name, password)
    
    if vm_structure is not None:
        display(vm_structure)
        
    # Пример данных
    print('\n🔍 Пример данных из витрины:')
    sample_data = run_query('SELECT * FROM dashboards.flight_analytics LIMIT 5;', database_name, password)
    if sample_data is not None:
        display(sample_data)
else:
    print('❌ Ошибка при создании витрины')

✅ Витрина данных создана!
📊 Всего строк в витрине: 1045726

📋 Структура витрины flight_analytics:


,column_name,data_type,is_nullable
0,sale_date,date,YES
1,sale_datetime,timestamp with time zone,YES
2,ticket_class,character varying,YES
3,ticket_amount,numeric,YES
4,flight_id,integer,YES
5,departure_airport,character,YES
6,departure_city,text,YES
7,departure_airport_name,text,YES
8,arrival_airport,character,YES
9,arrival_city,text,YES



🔍 Пример данных из витрины:


,sale_date,sale_datetime,ticket_class,ticket_amount,flight_id,departure_airport,departure_city,departure_airport_name,arrival_airport,arrival_city,arrival_airport_name,direction,direction_cities,scheduled_departure,scheduled_arrival,aircraft_code,aircraft_model,flight_status
0,2017-06-21,2017-06-21 12:48:00+00:00,Economy,11700.0,14584,PEE,Perm,Bolshoye Savino Airport,VKO,Moscow,Vnukovo International Airport,PEE → VKO,Perm → Moscow,2017-07-28 15:55:00+00:00,2017-07-28 17:20:00+00:00,773,Boeing 777-300,Arrived
1,2017-06-21,2017-06-21 12:48:00+00:00,Comfort,19900.0,14584,PEE,Perm,Bolshoye Savino Airport,VKO,Moscow,Vnukovo International Airport,PEE → VKO,Perm → Moscow,2017-07-28 15:55:00+00:00,2017-07-28 17:20:00+00:00,773,Boeing 777-300,Arrived
2,2017-06-21,2017-06-21 12:48:00+00:00,Economy,16600.0,14602,PEE,Perm,Bolshoye Savino Airport,OVB,Novosibirsk,Tolmachevo Airport,PEE → OVB,Perm → Novosibirsk,2017-07-16 12:05:00+00:00,2017-07-16 14:15:00+00:00,319,Airbus A319-100,Arrived
3,2017-06-21,2017-06-21 12:48:00+00:00,Economy,11700.0,3490,VKO,Moscow,Vnukovo International Airport,PEE,Perm,Bolshoye Savino Airport,VKO → PEE,Moscow → Perm,2017-07-16 08:00:00+00:00,2017-07-16 09:25:00+00:00,773,Boeing 777-300,Arrived
4,2017-06-21,2017-06-21 12:48:00+00:00,Business,49700.0,14602,PEE,Perm,Bolshoye Savino Airport,OVB,Novosibirsk,Tolmachevo Airport,PEE → OVB,Perm → Novosibirsk,2017-07-16 12:05:00+00:00,2017-07-16 14:15:00+00:00,319,Airbus A319-100,Arrived


## 1. График по суммам продаж за каждый из дней. По оси х - дни, по оси y - сумма продаж за день. Добавить фильтр по классу билета (эконом, бизнес и т.д.)

In [18]:
res = run_query("""
    SELECT sale_date, ticket_class, SUM(ticket_amount) as revenue
    FROM dashboards.flight_analytics 
    GROUP BY sale_date, ticket_class
    ORDER BY sale_date, ticket_class
""", database_name, password)

display(res)

,sale_date,ticket_class,revenue
0,2017-06-21,Business,49700.0
1,2017-06-21,Comfort,19900.0
2,2017-06-21,Economy,372300.0
3,2017-06-22,Business,292100.0
4,2017-06-22,Economy,483200.0
...,...,...,...
162,2017-08-14,Comfort,12615000.0
163,2017-08-14,Economy,368054700.0
164,2017-08-15,Business,108172700.0
165,2017-08-15,Comfort,11037500.0


![Динамика выручки по дням и классам](data/task_1.png)

## 2. График, показывающий процентное соотношение мест каждого из классов. Добавить фильтр по классу и модели самолета.

In [19]:
res = run_query("""
    SELECT 
        ad.model->>'en' as aircraft_model,
        s.fare_conditions as ticket_class,
        COUNT(*) as seats_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY ad.model->>'en'), 2) as percentage
    FROM bookings.seats s
    JOIN bookings.aircrafts_data ad ON s.aircraft_code = ad.aircraft_code
    GROUP BY ad.model->>'en', s.fare_conditions
    ORDER BY aircraft_model, ticket_class
""", database_name, password)

display(res)

,aircraft_model,ticket_class,seats_count,percentage
0,Airbus A319-100,Business,20,17.24
1,Airbus A319-100,Economy,96,82.76
2,Airbus A320-200,Business,20,14.29
3,Airbus A320-200,Economy,120,85.71
4,Airbus A321-200,Business,28,16.47
5,Airbus A321-200,Economy,142,83.53
6,Boeing 737-300,Business,12,9.23
7,Boeing 737-300,Economy,118,90.77
8,Boeing 767-300,Business,30,13.51
9,Boeing 767-300,Economy,192,86.49


![Процентное соотношение мест каждого из классов с фильтром по классу и модели самолета](data/task_2.png)

## 3. График, показывающий среднюю цену билетов по каждому из ТОП 5 самых дорогих направлений. Добавить фильтр по направлению.

In [21]:
res = run_query("""
    SELECT 
        arrival_airport,
        arrival_city,
        departure_airport,
        departure_city,
        ROUND(AVG(ticket_amount), 2) as avg_price,
        COUNT(*) as tickets_count
    FROM dashboards.flight_analytics 
    GROUP BY arrival_airport, arrival_city, departure_airport, departure_city
    ORDER BY avg_price DESC
    LIMIT 5;
""", database_name, password)

display(res)

,arrival_airport,arrival_city,departure_airport,departure_city,avg_price,tickets_count
0,GDX,Magadan,MRV,Mineralnye Vody,90665.70,207
1,UUS,Yuzhno-Sakhalinsk,DME,Moscow,89413.82,2170
2,MRV,Mineralnye Vody,GDX,Magadan,89401.45,207
3,UUS,Yuzhno-Sakhalinsk,SVO,Moscow,88703.86,2667
4,SVO,Moscow,UUS,Yuzhno-Sakhalinsk,88496.38,2511


![Топ-5 самых дорогих направлений по средней цене билета с фильтром по направлению](data/task_3.png)

## 4. Вывести среднюю цену билетов по каждой модели самолета. Добавить фильтр по модели самолета.

In [22]:
res = run_query("""
    SELECT 
        aircraft_model,
        ROUND(AVG(ticket_amount),2) as avg_price,
        COUNT(*) as tickets_count
    FROM dashboards.flight_analytics 
    GROUP BY aircraft_model
    ORDER BY avg_price DESC

""", database_name, password)

display(res)

,aircraft_model,avg_price,tickets_count
0,Airbus A319-100,51201.69,52853
1,Boeing 767-300,35033.56,124774
2,Boeing 777-300,23765.76,144376
3,Boeing 737-300,16568.16,86102
4,Airbus A321-200,15291.51,107129
5,Sukhoi Superjet-100,13985.54,365698
6,Bombardier CRJ-200,13207.66,150122
7,Cessna 208 Caravan,6568.55,14672


![Топ-5 самых дорогих направлений по средней цене билета с фильтром по направлению](data/task_4.png)

## Анализ аномально высокой стоимости билетов на Airbus A319-100

**Наблюдение:** Airbus A319-100 показывает самую высокую среднюю цену билета (51201.69 руб) среди всех моделей самолетов при значительном объеме продаж (52853 билета).

In [9]:
a319_analysis = run_query("""
    SELECT 
        departure_airport || ' → ' || arrival_airport as direction,
        departure_city || ' → ' || arrival_city as direction_cities,
        route_avg_price,
        route_tickets,
        a319_tickets,
        ROUND(a319_tickets * 100.0 / route_tickets, 1) as a319_share_percent
    FROM (
        SELECT 
            departure_airport,
            arrival_airport,
            departure_city,
            arrival_city,
            ROUND(AVG(ticket_amount), 2) as route_avg_price,
            COUNT(*) as route_tickets,
            COUNT(CASE WHEN aircraft_model = 'Airbus A319-100' THEN 1 END) as a319_tickets
        FROM dashboards.flight_analytics 
        GROUP BY departure_airport, arrival_airport, departure_city, arrival_city
        HAVING AVG(ticket_amount) > 50000
    ) AS routes
    ORDER BY route_avg_price DESC;
""", database_name, password)

print('📊 A319 НА САМЫХ ДОРОГИХ НАПРАВЛЕНИЯХ:')
display(a319_analysis)

📊 A319 НА САМЫХ ДОРОГИХ НАПРАВЛЕНИЯХ:


,direction,direction_cities,route_avg_price,route_tickets,a319_tickets,a319_share_percent
0,MRV → GDX,Mineralnye Vody → Magadan,90665.70,207,0,0.0
1,DME → UUS,Moscow → Yuzhno-Sakhalinsk,89413.82,2170,2170,100.0
2,GDX → MRV,Magadan → Mineralnye Vody,89401.45,207,0,0.0
3,SVO → UUS,Moscow → Yuzhno-Sakhalinsk,88703.86,2667,2667,100.0
4,UUS → SVO,Yuzhno-Sakhalinsk → Moscow,88496.38,2511,2511,100.0
5,DME → PKC,Moscow → Petropavlovsk,88383.37,1479,0,0.0
6,PKC → DME,Petropavlovsk → Moscow,88191.43,1377,0,0.0
7,UUS → DME,Yuzhno-Sakhalinsk → Moscow,88105.28,2027,2027,100.0
8,SVO → DYR,Moscow → Anadyr,82360.73,303,303,100.0
9,VKO → VVO,Moscow → Vladivostok,82178.88,1899,0,0.0


**Вывод**: Высокая средняя цена билета на A319-100 (51202 руб) объясняется не стоимостью перелета на этом самом самолете, а его стопроцентным использованием на множестве самых дорогих направлений сети, где средние цены билетов достигают 90665.70
руб.